In [1]:
%matplotlib inline
%load_ext memory_profiler

import os,time,datetime,sys,pickle
import scanpy as sc
from scipy import sparse
from sklearn.model_selection import train_test_split
import numpy as np

sc.settings.verbosity=2

## Transduction

Need masking indices

In [ ]:
# load data
pdfp = '/home/ngr4/project/scgraph/data/processed/'

if True :
    start=time.time()
    fname='transduction_50p.h5ad'
    backed=None # None if not, 'r+' if want to modify AnnData
    %memit adata = sc.read_h5ad(os.path.join(pdfp,fname),backed=backed)
    print('Data obj loaded in {:.2f}-s @'.format(time.time()-start)+datetime.datetime.now().strftime('%y%m%d.%H:%M:%S'))

In [ ]:
%memit adj=adata.uns['neighbors']['connectivities']+sparse.diags([1]*adata.shape[0]).tocsr()
%memit features = (adata.X - adata.X.min()) / (adata.X.max() - adata.X.min())


In [ ]:
# masking
idx_train,idx_test = train_test_split(range(adata.shape[0]), test_size=0.2, random_state=42, stratify=adata.obs['MS'])
idx_test,idx_val = train_test_split(idx_test, test_size=0.5, random_state=42, stratify=adata.obs['MS'][idx_test])
train_mask = [1 if i in idx_train else 0 for i in range(node_features.shape[0])]
val_mask = [1 if i in idx_val else 0 for i in range(node_features.shape[0])]
test_mask = [1 if i in idx_test else 0 for i in range(node_features.shape[0])]

In [ ]:
# lexport
pdfp = '/home/ngr4/project/scgraph/data/processed/'
        
with open(os.path.join(pdfp,'transduction_50pData.pkl'),'wb') as f :
    d = {'features':features,
         'adj':adj,
         'labels':adata.obs['MS'],
         'train_mask':train_mask,
         'val_mask':val_mask,
         'test_mask':test_mask}
    pickle.dump(d, f, protocol=pickle.HIGHEST_PROTOCOL)
    f.close()
    del d